names: Subrat Kishore Dutta , Prathvish Mithare
studentID: 7028082, 7028692
email: subratkishoredutta1234@gmail.com, prathvishmithare7@gmail.com

## Exercise 2

In [719]:
import numpy as np

np.random.seed(0)

def matrixfunc():
    random = np.random.rand(4,4)
    trans = random.transpose()
    inverse = np.linalg.inv(trans)
    print(inverse)
    return inverse
def eigmat():
    random = np.random.rand(4,4)
    w, v = np.linalg.eig(random)
    print("eigen value",w)
    print("eigen vector",v)
    return w,v

In [720]:
matrixfunc()
eigmat()

[[-2.93661054  1.66428382  3.929769   -1.73862477]
 [ 0.62452847 -0.43068264 -1.95008776  2.09349729]
 [ 1.79269745 -1.077603   -0.51994879  0.18396553]
 [ 1.09060329  0.54137516 -1.46025687 -0.19368226]]
eigen value [ 2.34634463+0.j         -0.41335836+0.29643305j -0.41335836-0.29643305j
  0.21733603+0.j        ]
eigen vector [[ 0.5140693 +0.j          0.24158654-0.50695888j  0.24158654+0.50695888j
  -0.07233623+0.j        ]
 [ 0.64668786+0.j         -0.41627784+0.23902478j -0.41627784-0.23902478j
  -0.7967314 +0.j        ]
 [ 0.3898993 +0.j          0.65218427+0.j          0.65218427-0.j
   0.20286588+0.j        ]
 [ 0.4068244 +0.j         -0.13260429+0.10620892j -0.13260429-0.10620892j
   0.56465208+0.j        ]]


(array([ 2.34634463+0.j        , -0.41335836+0.29643305j,
        -0.41335836-0.29643305j,  0.21733603+0.j        ]),
 array([[ 0.5140693 +0.j        ,  0.24158654-0.50695888j,
          0.24158654+0.50695888j, -0.07233623+0.j        ],
        [ 0.64668786+0.j        , -0.41627784+0.23902478j,
         -0.41627784-0.23902478j, -0.7967314 +0.j        ],
        [ 0.3898993 +0.j        ,  0.65218427+0.j        ,
          0.65218427-0.j        ,  0.20286588+0.j        ],
        [ 0.4068244 +0.j        , -0.13260429+0.10620892j,
         -0.13260429-0.10620892j,  0.56465208+0.j        ]]))

## Exercise 3 

In [721]:
## importing the required libraries
import torch
import cv2
import os
import numpy as np
import math
import torchvision
from PIL import Image
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision.datasets as datasets
from torchvision.transforms.functional import resize 
from torch.utils.data import Dataset,DataLoader

In [722]:
## defining the customdataset
class customDataset(Dataset):
    def __init__(self,image_paths,labels):
        self.labels = labels
        self.image_paths=image_paths 
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self,id,transform=None):
        label = self.labels[id]
        image = cv2.cvtColor(cv2.imread(self.image_paths[id]),cv2.COLOR_BGR2RGB)
        image=np.transpose(image,(2,0,1)).astype(np.float32)#(2,0,1) changes the format to (channel,width,height) from (width,height,channels)
        return torch.tensor(image),label#torch.tensor(image),


class customDataLoader(DataLoader):
    def __init__(self,customdata,batch_size,collate_fn=None):
        self.customdata=customdata
        self.batch_size=batch_size
        self.collate_fn=collate_fn

    def loader(self):
        if collate_fn!=None:
            return DataLoader(self.customdata,self.batch_size,collate_fn=self.collate_fn)
        elif collate_fn == None:
            return DataLoader(self.customdata,self.batch_size)

def custom_collate(batch):
    instance=batch[0][0].size()[2]
    h_resized=int(instance*0.8)
    w_resized=int(instance*0.8)
    image = [torchvision.transforms.Resize((int(h_resized),int(w_resized)))(sample[0]) for sample in batch] #resizing the images
    b = torch.Tensor(1,3, h_resized, w_resized)
    image=torch.stack(image, out=b) #convering the list of tensors into a single tensors
    labels = [sample[1] for sample in batch] 
    return image,labels


In [723]:
## we are using a custom dataset from our local system with its local path
base='D:/Saarland University/NNTI/data/' # path to the image directory
images=os.listdir('D:/Saarland University/NNTI/data') # list of the image file names
for i in range(len(images)):
    images[i]=base+images[i] ## paths of the images in the local system 
labels=[]
for i in range(1000):
    labels.append(0) #defining dummy lebels 

customdata=customDataset(images,labels)# loading the custom data

#creating objects of the class customDataLoader with the custom_collate function.
customloader1=customDataLoader(customdata,batch_size=10,collate_fn=custom_collate)
#creating objects of the class customDataLoader without the custom_collate function.
customloader2=customDataLoader(customdata,batch_size=10,collate_fn=None)

In [724]:
#unpacking the data with a batch size of 10 with the original size of the image
x,y=next(iter(customloader2.loader()))
print(x.size())

torch.Size([10, 3, 512, 512])


In [725]:
#unpacking the data with a batch size of 10 with the 80% of the original size of the image
x,y=next(iter(customloader1.loader()))
print(x.size())

C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: UserWarning: An output with one or more elements was resized since it had shape [1, 3, 409, 409], which does not match the required output shape [10, 3, 409, 409].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\Resize.cpp:24.)


torch.Size([10, 3, 409, 409])


In [727]:
x[0].size()

torch.Size([3, 409, 409])